משימות:
1. לייצר פיצ'רים מטמטיים מהקטגוריות והתקצירים - בוצע
2.  לחבר אותם לאינדקסים של הטבלה לפי הקטגוריות - בוצע
3. ליצור קובץ שלהם- בוצע
4. להכניס אותם למודל כשכבה נפרדת, להבין כיצד נכון לשלב את זה - בוצע 
5. לחזות - בוצע
6. ליצור מודל "ברירת מחדל" שממליץ על פופולריות
7. לראות איך משבים ריקול ופרפ גם למודל הזה
8. לשקול לסנן קוראים עם פחות מ10 ספרים
9. להוסיף למצגת

In [1]:
import math, pydot, graphviz
import numpy as np
import pandas as pd
from collections import defaultdict
# Split the data into train and test sets
# importing relevant libraries
from sklearn.model_selection import train_test_split

In [2]:
import keras
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils import plot_model
from keras.utils import model_to_dot
keras.utils.pydot = pydot
#plot_model(model)

In [3]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

In [4]:
def reader_encode(df,y=None):

    cat_var = ['Language','country']
    one_hot = OneHotEncoder(sparse=False)  # , drop = 'first')
    encoder_var_array = one_hot.fit_transform(df[cat_var])
    encoder_name = one_hot.get_feature_names_out(cat_var)
    encoder_vars_df = pd.DataFrame(encoder_var_array, columns=encoder_name)
    df = pd.concat([df, encoder_vars_df], axis=1)
    return df

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer


יש לנו בעיה בשימוש במשתנים האלה, יש הרבה מאוד ספרים שאין להם קטגוריה, אפשר להזין להם 0, אבל זה עדיין יהיה בעל משמעות מסוימת וזה לא יהיה נכון

In [6]:
df = pd.read_csv(r'C:\Users\yotam\Desktop\naya\df_clean_after_pca')
df = df[df['rating']!=0]
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])
df = df.reset_index(drop = True)

In [8]:
df.head()

,Unnamed: 0,index,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,...,221,222,223,224,225,226,227,228,229,isbn_num
0,0,0,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0
1,2,2,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0
2,4,4,67544,"toronto, ontario, canada",30.0000,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0
3,7,7,116866,"ottawa, ,",34.7439,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0
4,9,9,200273,"comber, ontario, canada",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0


In [7]:
book_author = df['book_author']
num_dict = {}
for i,j in enumerate(book_author.unique()):
    num_dict[j]=i
df['author_num'] = df['book_author'].apply(lambda txt: num_dict[txt])


In [8]:
df =reader_encode(df)
#לשקול לעשות פה שאפל מוגדר
train_users, test_users = train_test_split(df, test_size = 0.2 , random_state= 42)
x_train = train_users.drop(columns=['rating'])
y_train = train_users["rating"]

x_test = test_users.drop(columns=['rating'])
y_test = test_users["rating"]
all_users = train_users["user_id"].unique()
all_books = train_users["isbn_num"].unique()
all_authors = train_users["author_num"].unique()


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
#df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

In [5]:
#df = pd.read_csv(r"C:\Users\yotam\Desktop\naya\Preprocessed_data.csv")

## Implemation of TenserFlow

### naive model

# full pipline

In [16]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(flatten)

model = tf.keras.Model(inputs=[user_input, book_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 integer_lookup_2 (IntegerL  (None, 1)                    0         ['user[0][0]']                
 ookup)                                                                                           
                                                                                                  
 integer_lookup_3 (IntegerL  (None, 1)                    0         ['book[0][0]']          

In [18]:
 model.fit(
     x={
         "user": x_train["user_id"],
         "book": x_train["isbn_num"]
     },
     y=y_train.values,
     batch_size=128,
     epochs=20,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
     ],
 )


Epoch 1/20
1164/1164 [==============================] - 12s 10ms/step - loss: 7.4251 - mean_absolute_error: 2.3758 - root_mean_squared_error: 2.7249 - val_loss: 6.8057 - val_mean_absolute_error: 2.2563 - val_root_mean_squared_error: 2.6088
Epoch 2/20
1164/1164 [==============================] - 12s 10ms/step - loss: 6.3311 - mean_absolute_error: 2.1628 - root_mean_squared_error: 2.5162 - val_loss: 5.9958 - val_mean_absolute_error: 2.0909 - val_root_mean_squared_error: 2.4486
Epoch 3/20
1164/1164 [==============================] - 11s 9ms/step - loss: 5.5415 - mean_absolute_error: 1.9950 - root_mean_squared_error: 2.3540 - val_loss: 5.4209 - val_mean_absolute_error: 1.9672 - val_root_mean_squared_error: 2.3283
Epoch 4/20
1164/1164 [==============================] - 12s 10ms/step - loss: 4.9245 - mean_absolute_error: 1.8545 - root_mean_squared_error: 2.2191 - val_loss: 4.9960 - val_mean_absolute_error: 1.8713 - val_root_mean_squared_error: 2.2352
Epoch 5/20
1164/1164 [===================

In [19]:
model.evaluate({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"]#.reshape(-1, 1)
},y_test)

1293/1293 [==============================] - 3s 1ms/step - loss: 3.7359 - mean_absolute_error: 1.5488 - root_mean_squared_error: 1.9328


[3.7358641624450684, 1.5488183498382568, 1.9328383207321167]

 ### model with more inputs:

 ### model with emmbeding category and summerys:

In [33]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)




feature_input = tf.keras.layers.Input(shape=(230,), name="user_book_data")
feature = tf.keras.layers.Dense(200, name="add_some_feature")(feature_input)
dropout1 = tf.keras.layers.Dropout(0.3, name="drop_some_feature")(feature)
feature2 = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature2")(dropout1)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
con = tf.keras.layers.concatenate([flatten,feature2])
dense2 = tf.keras.layers.Dense(128, activation = 'relu', name="dence_layer2")(con)
dropout2 = tf.keras.layers.Dropout(0.3, name="drop_some_feature3")(dense2)
dense3 = tf.keras.layers.Dense(64, activation = 'relu', name="dence_layer3")(dropout2)
dropout3 = tf.keras.layers.Dropout(0.3, name="drop_some_feature4")(dense3)
dense4 = tf.keras.layers.Dense(16, activation = 'relu', name="dence_layer4")(dropout3)
dropout4 = tf.keras.layers.Dropout(0.3, name="drop_some_feature5")(dense4)

squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(dropout4)


#dropout2 = tf.keras.layers.Dropout(0.25, name="drop_some_feature2")(dropout2)



#dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
#add = tf.keras.layers.Add()([dot, user_bias, book_bias])
#flatten = tf.keras.layers.Flatten()(add)
#con = tf.keras.layers.concatenate([flatten,dropout2])
#dense2 = tf.keras.layers.Dense(100, activation = 'relu', name="dence_layer")(con)
#dropout2 = tf.keras.layers.Dropout(0.2, name="drop_some_feature3")(dense2)

#squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(con)

model = tf.keras.Model(inputs=[user_input, book_input,feature_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [34]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 integer_lookup_4 (IntegerL  (None, 1)                    0         ['user[0][0]']                
 ookup)                                                                                           
                                                                                                  
 integer_lookup_5 (IntegerL  (None, 1)                    0         ['book[0][0]']          

In [35]:
 model.fit(
     x={
         "user": x_train["user_id"],
         "book": x_train["isbn_num"],
         "user_book_data":x_train[[str(i) for i in range(0,230)]]
     },
     y=y_train.values,
     batch_size=128,
     epochs=30,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)
     ],
 )


Epoch 1/30
1164/1164 [==============================] - 23s 16ms/step - loss: 4.6601 - mean_absolute_error: 1.7444 - root_mean_squared_error: 2.1587 - val_loss: 3.1517 - val_mean_absolute_error: 1.4580 - val_root_mean_squared_error: 1.7753
Epoch 2/30
1164/1164 [==============================] - 12s 11ms/step - loss: 4.2731 - mean_absolute_error: 1.6274 - root_mean_squared_error: 2.0671 - val_loss: 2.9671 - val_mean_absolute_error: 1.3936 - val_root_mean_squared_error: 1.7225
Epoch 3/30
1164/1164 [==============================] - 13s 11ms/step - loss: 3.9745 - mean_absolute_error: 1.5246 - root_mean_squared_error: 1.9936 - val_loss: 2.9886 - val_mean_absolute_error: 1.3944 - val_root_mean_squared_error: 1.7288
Epoch 4/30
1164/1164 [==============================] - 13s 11ms/step - loss: 3.6755 - mean_absolute_error: 1.4162 - root_mean_squared_error: 1.9171 - val_loss: 3.0045 - val_mean_absolute_error: 1.3808 - val_root_mean_squared_error: 1.7333
Epoch 5/30
1164/1164 [==================

In [69]:
model.save('deep_model_emmbeding.h5')


C:\Users\yotam\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [36]:
model.evaluate({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"],#.reshape(-1, 1),
    "user_book_data":x_test[[str(i) for i in range(0,230)]]
},y_test)

1293/1293 [==============================] - 3s 2ms/step - loss: 3.0212 - mean_absolute_error: 1.4065 - root_mean_squared_error: 1.7382


[3.021188497543335, 1.4065338373184204, 1.7381566762924194]

In [54]:
pred = model.predict({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"],#.reshape(-1, 1),
    "user_book_data":x_test[[str(i) for i in range(0,230)]]
})


1293/1293 [==============================] - 2s 1ms/step


In [68]:
minp = 99
maxp =0
for i in range(len(pred)):
    for j in range(len(pred[i])):
        minp = min(minp,pred[i][j])
        maxp = max(maxp,pred[i][j])
print(minp,",",maxp)

5.5 , 9.991598


### with country and lang

In [10]:
list(x_train.columns)[250]

'author_num'

In [25]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)




feature_input = tf.keras.layers.Input(shape=(230,), name="book_data")
feature = tf.keras.layers.Dense(200, name="add_some_feature")(feature_input)
dropout1 = tf.keras.layers.Dropout(0.3, name="drop_some_feature")(feature)
feature2 = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature2")(dropout1)

user_feature_input = tf.keras.layers.Input(shape=(176,), name="user_data")
feature_u = tf.keras.layers.Dense(128, name="add_some_feature_u")(user_feature_input)
dropout1_u = tf.keras.layers.Dropout(0.4, name="drop_some_feature_u")(feature_u)
feature2_u = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature2_u")(dropout1_u)


dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
con_book_users = tf.keras.layers.concatenate([feature2,feature2_u])
feature2_u = tf.keras.layers.Dense(128, activation = 'relu', name="reader_book_type")(con_book_users)

con = tf.keras.layers.concatenate([flatten,feature2_u])#,feature2_u])
dense2 = tf.keras.layers.Dense(128, activation = 'relu', name="dence_layer2")(con)
dropout2 = tf.keras.layers.Dropout(0.3, name="drop_some_feature3")(dense2)
dense3 = tf.keras.layers.Dense(64, activation = 'relu', name="dence_layer3")(dropout2)
dropout3 = tf.keras.layers.Dropout(0.3, name="drop_some_feature4")(dense3)
dense4 = tf.keras.layers.Dense(16, activation = 'relu', name="dence_layer4")(dropout3)
dropout4 = tf.keras.layers.Dropout(0.3, name="drop_some_feature5")(dense4)

squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(dropout4)



model = tf.keras.Model(inputs=[user_input, book_input,feature_input,user_feature_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book_data (InputLayer)      [(None, 230)]                0         []                            
                                                                                                  
 user_data (InputLayer)      [(None, 176)]                0         []                            
                                                                                              

['Language_9',
 'Language_da',
 'Language_de',
 'Language_en',
 'Language_es',
 'Language_fr',
 'Language_it',
 'Language_nl',
 'Language_ru',
 'Language_th',
 'country_,',
 'country_aberdeenshire, united kingdom',
 'country_albania',
 'country_alderney',
 'country_america',
 'country_andalucia, spain',
 'country_antarctica',
 'country_argentina',
 'country_aruba',
 'country_australia',
 'country_austria',
 'country_bahamas',
 'country_bahrain',
 'country_barbados',
 'country_belgium',
 'country_belize',
 'country_bermuda',
 'country_bolivia',
 'country_brazil',
 'country_british columbia, canada',
 'country_bulgaria',
 'country_burma',
 'country_california, usa',
 'country_cambridgeshire, united kingdom',
 'country_canada',
 'country_canary islands, spain',
 'country_cape verde',
 'country_catalonia',
 'country_catalunya, spain',
 'country_cayman islands',
 'country_chile',
 'country_china',
 'country_co. carlow, ireland',
 'country_costa rica',
 'country_cuba',
 'country_cyprus',
 'c

In [26]:
 model.fit(
     x={
         "user": x_train["user_id"],
         "book": x_train["isbn_num"],
         "book_data":x_train[[str(i) for i in range(0,230)]], #category and summary
         'user_data':x_train[list(x_train.columns)[251:]]#+['age','year_of_publication']] #
     },
     y=y_train.values,
     batch_size=128,
     epochs=100,
     validation_split=0.1, # for early stopping
     shuffle= True,

     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=13, restore_best_weights=True)
     ],
 )


Epoch 1/100
1164/1164 [==============================] - 18s 14ms/step - loss: 4.6415 - mean_absolute_error: 1.7400 - root_mean_squared_error: 2.1544 - val_loss: 3.2592 - val_mean_absolute_error: 1.4913 - val_root_mean_squared_error: 1.8053
Epoch 2/100
1164/1164 [==============================] - 21s 18ms/step - loss: 4.2664 - mean_absolute_error: 1.6238 - root_mean_squared_error: 2.0655 - val_loss: 3.1716 - val_mean_absolute_error: 1.4629 - val_root_mean_squared_error: 1.7809
Epoch 3/100
1164/1164 [==============================] - 14s 12ms/step - loss: 3.9641 - mean_absolute_error: 1.5192 - root_mean_squared_error: 1.9910 - val_loss: 3.0105 - val_mean_absolute_error: 1.3966 - val_root_mean_squared_error: 1.7351
Epoch 4/100
1164/1164 [==============================] - 15s 13ms/step - loss: 3.6471 - mean_absolute_error: 1.4065 - root_mean_squared_error: 1.9097 - val_loss: 3.1164 - val_mean_absolute_error: 1.4227 - val_root_mean_squared_error: 1.7653
Epoch 5/100
1164/1164 [=============

In [27]:
model.evaluate({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"],#.reshape(-1, 1),
    "book_data":x_test[[str(i) for i in range(0,230)]],
    'user_data':x_test[list(x_train.columns)[251:]]
},y_test)

1293/1293 [==============================] - 3s 2ms/step - loss: 3.0826 - mean_absolute_error: 1.4157 - root_mean_squared_error: 1.7557


[3.082643747329712, 1.4157321453094482, 1.7557458877563477]

In [28]:
import numpy as np
from keras import Input, Model
from keras.layers import SimpleRNN, Dense


In [32]:
model.save('deep_model_emmbeding_leng.h5')


C:\Users\yotam\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
model.evaluate({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"],#.reshape(-1, 1),
    "user_book_data":x_test[[str(i) for i in range(0,230)]],
    'user_data_add':x_test[list(x_train.columns)[250:]]
},y_test)

1293/1293 [==============================] - 3s 2ms/step - loss: 3.0913 - mean_absolute_error: 1.4042 - root_mean_squared_error: 1.7582


[3.0912551879882812, 1.404232144355774, 1.7581965923309326]

## with outhers emmbeded

In [20]:
df.head()

,Unnamed: 0,index,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,...,221,222,223,224,225,226,227,228,229,isbn_num
0,0,0,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0
1,2,2,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0
2,4,4,67544,"toronto, ontario, canada",30.0000,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0
3,7,7,116866,"ottawa, ,",34.7439,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0
4,9,9,200273,"comber, ontario, canada",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,...,0.03518,-0.125755,-0.019404,0.044569,-0.040087,0.005104,0.023391,0.116895,0.013755,0


In [22]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

author_input = tf.keras.layers.Input(shape=(1,), name="author")
author_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_authors)(author_input)
author_embedding = tf.keras.layers.Embedding(input_dim=len(all_authors) + 1, output_dim=32)(author_as_integer)

dot_author = tf.keras.layers.Dot(axes=2)([user_embedding, author_embedding])
flatten_author = tf.keras.layers.Flatten()(dot_author)


feature_input = tf.keras.layers.Input(shape=(230,), name="book_data")
feature = tf.keras.layers.Dense(200, name="add_some_feature")(feature_input)
dropout1 = tf.keras.layers.Dropout(0.3, name="drop_some_feature")(feature)
feature2 = tf.keras.layers.Dense(128, activation = 'relu', name="add_some_feature2")(dropout1)
dropout1_2 = tf.keras.layers.Dropout(0.3, name="drop_some_feature22")(feature2)
feature2_2 = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature22")(dropout1_2)

user_feature_input = tf.keras.layers.Input(shape=(176,), name="user_data")
feature_u = tf.keras.layers.Dense(128, name="add_some_feature_u")(user_feature_input)
dropout1_u = tf.keras.layers.Dropout(0.4, name="drop_some_feature_u2")(feature_u)
feature2_u = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature2_u2")(dropout1_u)


dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
con_book_users = tf.keras.layers.concatenate([feature2_2,feature2_u])
feature2_u = tf.keras.layers.Dense(128, activation = 'relu', name="reader_book_type")(con_book_users)

con = tf.keras.layers.concatenate([flatten,flatten_author,feature2_u])#,feature2_u])
dense2 = tf.keras.layers.Dense(128, activation = 'relu', name="dence_layer2")(con)
dropout2 = tf.keras.layers.Dropout(0.3, name="drop_some_feature3")(dense2)
dense3 = tf.keras.layers.Dense(64, activation = 'relu', name="dence_layer3")(dropout2)
dropout3 = tf.keras.layers.Dropout(0.3, name="drop_some_feature4")(dense3)
dense4 = tf.keras.layers.Dense(16, activation = 'relu', name="dence_layer4")(dropout3)
dropout4 = tf.keras.layers.Dropout(0.3, name="drop_some_feature5")(dense4)
dense5 = tf.keras.layers.Dense(8, activation = 'relu', name="dence_layer5")(dropout4)
dropout5 = tf.keras.layers.Dropout(0.3, name="drop_some_feature6")(dense5)

squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(dropout5)



model = tf.keras.Model(inputs=[user_input, book_input,author_input,feature_input,user_feature_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [18]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 book_data (InputLayer)      [(None, 230)]                0         []                            
                                                                                                  
 add_some_feature (Dense)    (None, 200)                  46200     ['book_data[0][0]']           
                                                                                                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book (InputLayer)           [(None, 1)]                  0         []                            
                                                                                            

 t)                                                                                               
                                                                                                  
 dence_layer4 (Dense)        (None, 16)                   1040      ['drop_some_feature4[0][0]']  
                                                                                                  
 drop_some_feature5 (Dropou  (None, 16)                   0         ['dence_layer4[0][0]']        
 t)                                                                                               
                                                                                                  
 dence_layer5 (Dense)        (None, 8)                    136       ['drop_some_feature5[0][0]']  
                                                                                                  
 drop_some_feature6 (Dropou  (None, 8)                    0         ['dence_layer5[0][0]']        
 t)       

In [23]:
 model.fit(
     x={
         "user": x_train["user_id"],
         "book": x_train["isbn_num"],
         "author":x_train['author_num'],
         "book_data":x_train[[str(i) for i in range(0,230)]], #category and summary
         'user_data':x_train[list(x_train.columns)[251:]]#+['age','year_of_publication']] #
     },
     y=y_train.values,
     batch_size=128,
     epochs=100,
     validation_split=0.1, # for early stopping
     shuffle= True,
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
     ],
 )
# לשקול לעשות פה שאפל מוגדר בוולידיישן, יש בעיה עם זה שלא עושים שאפל


Epoch 1/100
1164/1164 [==============================] - 28s 22ms/step - loss: 4.7571 - mean_absolute_error: 1.7696 - root_mean_squared_error: 2.1811 - val_loss: 3.3345 - val_mean_absolute_error: 1.5144 - val_root_mean_squared_error: 1.8261
Epoch 2/100
1164/1164 [==============================] - 17s 14ms/step - loss: 4.3159 - mean_absolute_error: 1.6420 - root_mean_squared_error: 2.0775 - val_loss: 3.0891 - val_mean_absolute_error: 1.4390 - val_root_mean_squared_error: 1.7576
Epoch 3/100
1164/1164 [==============================] - 19s 16ms/step - loss: 3.8654 - mean_absolute_error: 1.4947 - root_mean_squared_error: 1.9661 - val_loss: 2.8875 - val_mean_absolute_error: 1.3546 - val_root_mean_squared_error: 1.6993
Epoch 4/100
1164/1164 [==============================] - 17s 15ms/step - loss: 3.4520 - mean_absolute_error: 1.3535 - root_mean_squared_error: 1.8580 - val_loss: 3.1410 - val_mean_absolute_error: 1.4060 - val_root_mean_squared_error: 1.7723
Epoch 5/100
1164/1164 [=============

In [24]:
model.evaluate({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"],#.reshape(-1, 1),
    "author":x_test['author_num'],
    "book_data":x_test[[str(i) for i in range(0,230)]],
    'user_data':x_test[list(x_train.columns)[251:]]
},y_test)

1293/1293 [==============================] - 3s 2ms/step - loss: 2.9318 - mean_absolute_error: 1.3649 - root_mean_squared_error: 1.7122


[2.9317548274993896, 1.3649042844772339, 1.712236762046814]

In [13]:
model.save('deep_model_emmbeding_author.h5')


C:\Users\yotam\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
list(x_train.columns)[250:]

['author_num',
 'Language_9',
 'Language_da',
 'Language_de',
 'Language_en',
 'Language_es',
 'Language_fr',
 'Language_it',
 'Language_nl',
 'Language_ru',
 'Language_th',
 'country_,',
 'country_aberdeenshire, united kingdom',
 'country_albania',
 'country_alderney',
 'country_america',
 'country_andalucia, spain',
 'country_antarctica',
 'country_argentina',
 'country_aruba',
 'country_australia',
 'country_austria',
 'country_bahamas',
 'country_bahrain',
 'country_barbados',
 'country_belgium',
 'country_belize',
 'country_bermuda',
 'country_bolivia',
 'country_brazil',
 'country_british columbia, canada',
 'country_bulgaria',
 'country_burma',
 'country_california, usa',
 'country_cambridgeshire, united kingdom',
 'country_canada',
 'country_canary islands, spain',
 'country_cape verde',
 'country_catalonia',
 'country_catalunya, spain',
 'country_cayman islands',
 'country_chile',
 'country_china',
 'country_co. carlow, ireland',
 'country_costa rica',
 'country_cuba',
 'count

In [ ]:
# 6831 real val, 3166 with "a book cover

In [4]:
car = pd.read_csv(r'cover_page.csv')

In [19]:
car[car['cover_page'].str.contains('a book cover')]

,Unnamed: 0,img_l,cover_page
0,0,http://images.amazon.com/images/P/0002005018.0...,a close up of a book cover with two women
5,5,http://images.amazon.com/images/P/0440234743.0...,a book cover of the testament
12,12,http://images.amazon.com/images/P/0345417623.0...,a book cover of time line by michael crichton
13,13,http://images.amazon.com/images/P/0312978383.0...,a book cover of winter solstice
15,15,http://images.amazon.com/images/P/0553264990.0...,a book cover of the last of the breed
...,...,...,...
7586,7586,http://images.amazon.com/images/P/0448095122.0...,a close up of a book cover with a woman in a tree
7587,7587,http://images.amazon.com/images/P/0451525264.0...,a book cover with a picture of a child holding...
7588,7588,http://images.amazon.com/images/P/0553272500.0...,a book cover of cheaper by the dozen
7597,7597,http://images.amazon.com/images/P/0816713170.0...,a book cover of a man leaning on a drum in the...
